In [1]:
# CSL Paper: Dimensional speech emotion recognition from acoustic and text
# Changelog:
# 2019-09-01: initial version
# 2019-10-06: optimizer MTL parameters with linear search (in progress)
# 2019-12-21: update file for CSL journal
#             this python use following feature: mean, std, silence

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE
from pytorch_pretrained_bert.modeling import BertModel
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from attention_helper import AttentionDecoder
#from read_csv import load_features

import random as rn
import tensorflow as tf

# import gensim
# from gensim.models import Word2Vec
# from gensim.models.keyedvectors import KeyedVectors
import codecs

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# x_text = np.load('../improv_loso/data/x_text.npy')

In [4]:
# x_text.shape

In [5]:
# load data - label
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_train = pd.read_csv(list_path_train, index_col=None)

# vad_train = [list_train['v'], list_train['a'], list_train['d']]
# vad_train = np.array(vad_train).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

## Text processing train
list_file = pd.concat([list_train, list_test])
text = list_file['transcript']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
nb_words = len(word_index) + 1
print('Found %s unique tokens' % len(word_index))

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_text = []
x_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# x_text = np.load('improv_loso/data/x_text.npy')
# x_train_text = x_text[:5744] 
# x_test_text = x_text[5744:]

# vad = np.load('improv_loso/data/vad.npy')
# vad_train = vad[:5744]
# vad_test = vad[5744:]
EMBEDDING_DIM = 768 #1024
# EMBEDDING_DIM = 300
# nb_words = 3438
# MAX_SEQUENCE_LENGTH = 300

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
# file_loc =  '../../../dimensionalSM/data/glove.42B.300d.txt'
# print (file_loc)

# gembeddings_index = {}
# with codecs.open(file_loc, encoding='utf-8') as f:
#     for line in f:
#         values = line.split(' ')
#         word = values[0]
#         gembedding = np.asarray(values[1:], dtype='float32')
#         gembeddings_index[word] = gembedding
# #
# f.close()
# print('G Word embeddings:', len(gembeddings_index))

# nb_words = len(word_index) +1
# g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
# for word, i in word_index.items():
#     gembedding_vector = gembeddings_index.get(word)
#     if gembedding_vector is not None:
#         g_word_embedding_matrix[i] = gembedding_vector
        
# print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))
def get_bert_embed_matrix():
    bert = BertModel.from_pretrained(BERT_FP)
    bert_embeddings = list(bert.children())[0]
    bert_word_embeddings = list(bert_embeddings.children())[0]
    mat = bert_word_embeddings.weight.data.numpy()
    return mat


def get_bert_embed_matrix():
    bert = BertModel.from_pretrained('bert-base-uncased')
    bert_embeddings = list(bert.children())[0]
    bert_word_embeddings = list(bert_embeddings.children())[0]
    mat = bert_word_embeddings.weight.data.numpy()
    return mat


g_word_embedding_matrix = get_bert_embed_matrix()


Found 3427 unique tokens


In [6]:
vad = np.load('../improv_loso/data/vad.npy')

In [7]:
# standardization
scaled_feature = False

# set Dropout
do = 0.3

# standardized label
scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad


# Concordance correlation coefficient (CCC)-based loss function
# using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [8]:
# split train/test
split = 5733
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)

In [9]:
# API model, if use RNN, first two rnn layer must return_sequences=True
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(*g_word_embedding_matrix.shape,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model1 = text_model1()

In [10]:
# main function for for LOSO
def main(time):
    model_1 = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [x_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [x_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict(x_text[4586:], batch_size=32)
    np.save('5733npy-BERT/text_5733-BERT-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 2580).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('5733npy-BERT/text-BERT_5733.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 164s 36ms/step - loss: 2.6778 - v_loss: 0.7897 - a_loss: 0.9283 - d_loss: 0.9504 - v_ccc: 0.2075 - a_ccc: 0.0680 - d_ccc: 0.0467 - val_loss: 2.6571 - val_v_loss: 0.8379 - val_a_loss: 0.9160 - val_d_loss: 0.9011 - val_v_ccc: 0.1611 - val_a_ccc: 0.0838 - val_d_ccc: 0.0981
Epoch 2/50
4586/4586 [==============================] - 159s 35ms/step - loss: 2.0089 - v_loss: 0.5626 - a_loss: 0.7008 - d_loss: 0.7487 - v_ccc: 0.4373 - a_ccc: 0.3006 - d_ccc: 0.2531 - val_loss: 2.7012 - val_v_loss: 0.8642 - val_a_loss: 0.9068 - val_d_loss: 0.9275 - val_v_ccc: 0.1344 - val_a_ccc: 0.0927 - val_d_ccc: 0.0717
Epoch 3/50
4586/4586 [==============================] - 159s 35ms/step - loss: 1.7691 - v_loss: 0.4876 - a_loss: 0.6179 - d_loss: 0.6632 - v_ccc: 0.5116 - a_ccc: 0.3817 - d_ccc: 0.3375 - val_loss: 2.6565 - val_v_loss

Epoch 11/50
1433/1433 [==============================] - 9s 6ms/step
[0.1316937953233719, 0.16310229897499084, 0.13337190449237823]
1
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 161s 35ms/step - loss: 2.7572 - v_loss: 0.8425 - a_loss: 0.9544 - d_loss: 0.9574 - v_ccc: 0.1553 - a_ccc: 0.0452 - d_ccc: 0.0423 - val_loss: 2.6700 - val_v_loss: 0.8617 - val_a_loss: 0.8999 - val_d_loss: 0.9061 - val_v_ccc: 0.1364 - val_a_ccc: 0.1002 - val_d_ccc: 0.0934
Epoch 2/50
4586/4586 [==============================] - 158s 34ms/step - loss: 2.0860 - v_loss: 0.5814 - a_loss: 0.7303 - d_loss: 0.7704 - v_ccc: 0.4177 - a_ccc: 0.2680 - d_ccc: 0.2282 - val_loss: 2.7162 - val_v_loss: 0.8557 - val_a_loss: 0.9168 - val_d_loss: 0.9411 - val_v_ccc: 0.1427 - val_a_ccc: 0.0828 - val_d_ccc: 0.0583
Epoch 3/50
4586/4586 [==============================] - 159s 35ms/step - loss: 1.7988 - v_loss: 0.5029 - a_loss: 0.6248 - d_loss: 0.6744 - v_ccc: 0.4994 - a_ccc: 0.

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4586/4586 [==============================] - 161s 35ms/step - loss: 1.0908 - v_loss: 0.2955 - a_loss: 0.3875 - d_loss: 0.4063 - v_ccc: 0.7045 - a_ccc: 0.6119 - d_ccc: 0.5928 - val_loss: 2.7866 - val_v_loss: 0.8599 - val_a_loss: 0.9296 - val_d_loss: 0.9950 - val_v_ccc: 0.1384 - val_a_ccc: 0.0703 - val_d_ccc: 0.0046
Epoch 10/50
4586/4586 [==============================] - 161s 35ms/step - loss: 1.0328 - v_loss: 0.2804 - a_loss: 0.3667 - d_loss: 0.3891 - v_ccc: 0.7212 - a_ccc: 0.6332 - d_ccc: 0.6129 - val_loss: 2.8013 - val_v_loss: 0.8764 - val_a_loss: 0.9288 - val_d_loss: 0.9946 - val_v_ccc: 0.1219 - val_a_ccc: 0.0715 - val_d_ccc: 0.0053
Epoch 11/50
4586/4586 [==============================] - 160s 35ms/step - loss: 0.9714 - v_loss: 0.2667 - a_loss: 0.3443 - d_loss: 0.3623 - v_ccc: 0.7354 - a_ccc: 0.6561 - d_ccc: 0.6371 - val_loss: 2.7459 - val_v_loss: 0.8607 - val_a_loss: 0.9067 - val_d_loss: 0.9762 - val_v_ccc: 0.1375 - val_a_ccc: 0.0932 - val_d_ccc: 0.0234
Epoch 12/50
4586/4586 [=====

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4586/4586 [==============================] - 160s 35ms/step - loss: 1.6085 - v_loss: 0.4392 - a_loss: 0.5628 - d_loss: 0.6050 - v_ccc: 0.5624 - a_ccc: 0.4356 - d_ccc: 0.3934 - val_loss: 2.7102 - val_v_loss: 0.8843 - val_a_loss: 0.8961 - val_d_loss: 0.9265 - val_v_ccc: 0.1138 - val_a_ccc: 0.1035 - val_d_ccc: 0.0726
Epoch 5/50
4586/4586 [==============================] - 165s 36ms/step - loss: 1.4273 - v_loss: 0.3899 - a_loss: 0.5044 - d_loss: 0.5338 - v_ccc: 0.6109 - a_ccc: 0.4961 - d_ccc: 0.4657 - val_loss: 2.6637 - val_v_loss: 0.8537 - val_a_loss: 0.8838 - val_d_loss: 0.9245 - val_v_ccc: 0.1456 - val_a_ccc: 0.1158 - val_d_ccc: 0.0748
Epoch 6/50
4586/4586 [==============================] - 164s 36ms/step - loss: 1.2898 - v_loss: 0.3506 - a_loss: 0.4616 - d_loss: 0.4813 - v_ccc: 0.6512 - a_ccc: 0.5399 - d_ccc: 0.5192 - val_loss: 2.6904 - val_v_loss: 0.8854 - val_a_loss: 0.8798 - val_d_loss: 0.9234 - val_v_ccc: 0.1141 - val_a_ccc: 0.1200 - val_d_ccc: 0.0755
Epoch 7/50
4586/4586 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4586/4586 [==============================] - 158s 34ms/step - loss: 1.2854 - v_loss: 0.3662 - a_loss: 0.4511 - d_loss: 0.4678 - v_ccc: 0.6339 - a_ccc: 0.5486 - d_ccc: 0.5322 - val_loss: 2.7744 - val_v_loss: 0.8808 - val_a_loss: 0.9221 - val_d_loss: 0.9693 - val_v_ccc: 0.1174 - val_a_ccc: 0.0780 - val_d_ccc: 0.0303
Epoch 7/50
4586/4586 [==============================] - 158s 34ms/step - loss: 1.1785 - v_loss: 0.3297 - a_loss: 0.4160 - d_loss: 0.4313 - v_ccc: 0.6702 - a_ccc: 0.5832 - d_ccc: 0.5681 - val_loss: 2.7272 - val_v_loss: 0.8608 - val_a_loss: 0.9114 - val_d_loss: 0.9521 - val_v_ccc: 0.1372 - val_a_ccc: 0.0884 - val_d_ccc: 0.0473
Epoch 8/50
4586/4586 [==============================] - 158s 34ms/step - loss: 1.1064 - v_loss: 0.3058 - a_loss: 0.3936 - d_loss: 0.4086 - v_ccc: 0.6951 - a_ccc: 0.6072 - d_ccc: 0.5912 - val_loss: 2.7476 - val_v_loss: 0.8631 - val_a_loss: 0.9208 - val_d_loss: 0.9616 - val_v_ccc: 0.1352 - val_a_ccc: 0.0794 - val_d_ccc: 0.0378
Epoch 9/50
4586/4586 [========

Epoch 5/50
4586/4586 [==============================] - 158s 35ms/step - loss: 1.4195 - v_loss: 0.3971 - a_loss: 0.4908 - d_loss: 0.5307 - v_ccc: 0.6025 - a_ccc: 0.5074 - d_ccc: 0.4706 - val_loss: 2.7030 - val_v_loss: 0.8720 - val_a_loss: 0.9113 - val_d_loss: 0.9170 - val_v_ccc: 0.1263 - val_a_ccc: 0.0887 - val_d_ccc: 0.0821
Epoch 6/50
4586/4586 [==============================] - 157s 34ms/step - loss: 1.3021 - v_loss: 0.3583 - a_loss: 0.4552 - d_loss: 0.4876 - v_ccc: 0.6415 - a_ccc: 0.5449 - d_ccc: 0.5115 - val_loss: 2.7065 - val_v_loss: 0.8352 - val_a_loss: 0.9115 - val_d_loss: 0.9569 - val_v_ccc: 0.1629 - val_a_ccc: 0.0881 - val_d_ccc: 0.0425
Epoch 7/50
4586/4586 [==============================] - 158s 34ms/step - loss: 1.1958 - v_loss: 0.3316 - a_loss: 0.4190 - d_loss: 0.4506 - v_ccc: 0.6697 - a_ccc: 0.5826 - d_ccc: 0.5519 - val_loss: 2.7169 - val_v_loss: 0.8441 - val_a_loss: 0.9138 - val_d_loss: 0.9568 - val_v_ccc: 0.1542 - val_a_ccc: 0.0864 - val_d_ccc: 0.0425
Epoch 8/50
4586/458

In [11]:
# hist1 = model1.fit(x_text[:split], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(x_text[split:], vad[split:].T.tolist())
# print(eval_metrik1[-3:])

# # make prediction
# predict = model1.predict(x_text[4596:], batch_size=32)
# np.save('text_glove_npy_loso', 
#          np.array(predict).reshape(3, 2570).T)